In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_validation/output/"
N_FOLDS <- 10

vst.counts <- readRDS(paste(OUT_DIR, "vst_counts.Rds", sep = ""))

vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
GEO_model_validation_tissue_detail.vec <-
   readRDS(paste(OUT_DIR, "GEO_model_validation_tissue_vec.Rds", sep = ""))
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls.Rds", sep = ""))
rxns <- rxn2ensembls.nls %>% names()

In [4]:
%%R
print(length(GEO_model_validation_tissue_detail.vec))
print(length(vst.count.mtx))
print(length(rxn2ensembls.nls))

[1] 1445
[1] 1445
[1] 7856


In [5]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [6]:
%%R
toi_indices <- seq(1,length(GEO_model_validation_tissue_detail.vec))
# print(toi_indices)

# filter annotations
GEO_model_validation_tissue_detail_vec_tis_of_interest <- GEO_model_validation_tissue_detail.vec[toi_indices]
length(GEO_model_validation_tissue_detail_vec_tis_of_interest)

[1] 1445


In [7]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
length(vst.count.mtx.tis_of_interest)

[1] 1445


In [8]:
%%R
training_indices <-
   caret::createDataPartition(
      GEO_model_validation_tissue_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
length(training_indices)

[1] 1445


In [9]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of dataprint(vst.count.mtx.train)
print(length(vst.count.mtx.train))
print(length(vst.count.mtx.test))

[1] 1445
[1] 0


In [10]:
%%R
GEO_model_validation_tissue_detail.vec.train <-
   GEO_model_validation_tissue_detail_vec_tis_of_interest[training_indices]
GEO_model_validation_tissue_detail.vec.test <-
   GEO_model_validation_tissue_detail_vec_tis_of_interest[-training_indices]

print(length(GEO_model_validation_tissue_detail.vec.train))
print(length(GEO_model_validation_tissue_detail.vec.test))

saveRDS(GEO_model_validation_tissue_detail.vec.train,file=paste(OUT_DIR,"GEO_model_validation_tissue_detial_vec_train.Rds",sep=""))

[1] 1445
[1] 0


In [11]:
%%R
cv_fold_indices <- caret::createFolds(GEO_model_validation_tissue_detail.vec.train,
                                      k = N_FOLDS)
print(length(cv_fold_indices))
binary_GEO_model_validation_tissue_annotations <- unique(GEO_model_validation_tissue_detail.vec)
print(length(binary_GEO_model_validation_tissue_annotations))

full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

[1] 10
[1] 13


In [12]:
%%R
print(length(rxns))

[1] 7856


In [13]:
%%R
n_rxns <- length(rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,".Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] "Processed 100 of 7856 reactions (1.3%)..."
[1] "Processed 200 of 7856 reactions (2.6%)..."
[1] "Processed 300 of 7856 reactions (3.8%)..."
[1] "Processed 400 of 7856 reactions (5.1%)..."
[1] "Processed 500 of 7856 reactions (6.4%)..."
[1] "Processed 600 of 7856 reactions (7.7%)..."
[1] "Processed 700 of 7856 reactions (8.9%)..."
[1] "Processed 800 of 7856 reactions (10.2%)..."
[1] "Processed 900 of 7856 reactions (11.5%)..."
[1] "Processed 1000 of 7856 reactions (12.7%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 7856 reactions (12.7%)..."
[1] "Processed 1100 of 7856 reactions (14%)..."
[1] "Processed 1200 of 7856 reactions (15.3%)..."
[1] "Processed 1300 of 7856 reactions (16.6%)..."
[1] "Processed 1400 of 7856 reactions (17.8%)..."
[1] "Processed 1500 of 7856 reactions (19.1%)..."
[1] "Processed 1600 of 7856 reactions (20.4%)..."
[1] "Processed 1700 of 7856 reactions (21.7%)..."
[1] "Processed 1800 of 7856 reactions (22.9%)..."
[1] "Processed 1900 of 7856 reactio

In [14]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()


           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6929433 370.1   10061882 537.4 10061882 537.4
Vcells 48760909 372.1   90221912 688.4 62499488 476.9


In [15]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train.Rds", sep = ""))

In [16]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0
   
   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      GEO_model_validation_tissue_detail.vec.train.cv_train <-
         GEO_model_validation_tissue_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      GEO_model_validation_tissue_detail.vec.train.cv_test <-
         GEO_model_validation_tissue_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_GEO_model_validation_tissue_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_GEO_model_validation_tissue_annotations) {
         binary_GEO_model_validation_tissue_detail_vec.test.cv_test_list[[tissue_annotation]] <-
            (GEO_model_validation_tissue_detail.vec.train.cv_test == tissue_annotation)
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = GEO_model_validation_tissue_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            GEO_model_validation_tissue_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_GEO_model_validation_tissue_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         
         tab <- table(cur_rxn_knn_calls,
                      binary_GEO_model_validation_tissue_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_GEO_model_validation_tissue_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-6798748 : Last ARI =  0.877299449596733 : Last ECOUNT =  55 : Last Lung MISCLASS =  0.00553265376541238 : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9840884 : Last ARI =  0.549410112726801 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0463597633005994 : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-2002466 : Last ARI =  0.52065093565355 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0297631720399788 : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-2247925 : Last ARI =  0.199708471675852 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.08988188793869 : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9016782 : Last ARI =  0.199708471675852 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.08988188793869 : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8871193 : Last ARI =  0.919241897776846 : Last ECOUNT =  65 : 

[1] "Last RXN_ID =  R-MMU-913529 : Last ARI =  0.0445783212198868 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0782790935997124 : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-190520 : Last ARI =  0.665669274619821 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0228521118382951 : Last Uterus MISCLASS =   . Now  94.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-5610767 : Last ARI =  0.815080359452256 : Last ECOUNT =  10 : Last Lung MISCLASS =  0.00625571207762988 : Last Uterus MISCLASS =   . Now  94 % remaining..."
[1] "Last RXN_ID =  R-MMU-5666169 : Last ARI =  0.847267627425762 : Last ECOUNT =  59 : Last Lung MISCLASS =  0.0103561698046818 : Last Uterus MISCLASS =   . Now  93.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-8852306 : Last ARI =  0.665669274619821 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0228521118382951 : Last Uterus MISCLASS =   . Now  93.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9668419 : Last ARI =  0.814530267891948 : Last ECOUNT =  11

[1] "Last RXN_ID =  R-MMU-500708 : Last ARI =  0.222820252771726 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0941345877086312 : Last Uterus MISCLASS =   . Now  88.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-421007 : Last ARI =  0.81628205874642 : Last ECOUNT =  8 : Last Lung MISCLASS =  0.00345326332985757 : Last Uterus MISCLASS =   . Now  88.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-983138 : Last ARI =  0.720274616821888 : Last ECOUNT =  9 : Last Lung MISCLASS =  0.016629871259654 : Last Uterus MISCLASS =   . Now  88.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-9818937 : Last ARI =  0.3627729316716 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0678483710049605 : Last Uterus MISCLASS =   . Now  88.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-202216 : Last ARI =  0.685295242538449 : Last ECOUNT =  9 : Last Lung MISCLASS =  0.0249493671340625 : Last Uterus MISCLASS =   . Now  88.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029272 : Last ARI =  0.387849115345604 : Last ECOUNT =  27 : Last

[1] "Last RXN_ID =  R-MMU-202375 : Last ARI =  0.248848942600067 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0512274314872331 : Last Uterus MISCLASS =   . Now  83.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-2730833 : Last ARI =  0.695172158544826 : Last ECOUNT =  32 : Last Lung MISCLASS =  0.0235512224139999 : Last Uterus MISCLASS =   . Now  82.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-430183 : Last ARI =  0.640580115860408 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0297354031700985 : Last Uterus MISCLASS =   . Now  82.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-5694423 : Last ARI =  0.73425199159871 : Last ECOUNT =  9 : Last Lung MISCLASS =  0.0180381837916084 : Last Uterus MISCLASS =   . Now  82.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9033516 : Last ARI =  0.767165102079036 : Last ECOUNT =  11 : Last Lung MISCLASS =  0.0104038095476452 : Last Uterus MISCLASS =   . Now  82.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-2032379 : Last ARI =  0.535870482245316 : Last ECOUNT =  2 :

[1] "Last RXN_ID =  R-MMU-8985900 : Last ARI =  0.598353547325783 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0235889514514928 : Last Uterus MISCLASS =   . Now  77.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-8987070 : Last ARI =  0.297085493645673 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0863563314165582 : Last Uterus MISCLASS =   . Now  77.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-8987230 : Last ARI =  0.154023697558789 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0823325008516317 : Last Uterus MISCLASS =   . Now  77.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-9618581 : Last ARI =  0.154023697558789 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0823325008516317 : Last Uterus MISCLASS =   . Now  77.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9701524 : Last ARI =  0.509736368872253 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.06431328402455 : Last Uterus MISCLASS =   . Now  77 % remaining..."
[1] "Last RXN_ID =  R-MMU-9835443 : Last ARI =  0.275986779696965 : Last ECOUNT =  2 : La

...

[1] "Last RXN_ID =  R-MMU-8956099 : Last ARI =  0.674397893671261 : Last ECOUNT =  7 : Last Lung MISCLASS =  0.0118163234601591 : Last Uterus MISCLASS =   . Now  27.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-5663114 : Last ARI =  0.0211040183715038 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0927358550970974 : Last Uterus MISCLASS =   . Now  26.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-879748 : Last ARI =  0.0211040183715038 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0927358550970974 : Last Uterus MISCLASS =   . Now  26.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-73806 : Last ARI =  0.250114420880826 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0919098202506327 : Last Uterus MISCLASS =   . Now  26.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-176517 : Last ARI =  0.0651138651536143 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0879399150513936 : Last Uterus MISCLASS =   . Now  26.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-265427 : Last ARI =  0.0606454821151348 : Last ECOUNT =  1 :

[1] "Last RXN_ID =  R-MMU-2424246 : Last ARI =  0.312533532342157 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.078862453723696 : Last Uterus MISCLASS =   . Now  21.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-197268 : Last ARI =  0.355877394159695 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.093367642427633 : Last Uterus MISCLASS =   . Now  21.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-8847427 : Last ARI =  0.0634309388548819 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0796402244436019 : Last Uterus MISCLASS =   . Now  21.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-2076220 : Last ARI =  0.369543582559957 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0374088095043457 : Last Uterus MISCLASS =   . Now  21.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9661723 : Last ARI =  0.336220123157315 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0830793823832322 : Last Uterus MISCLASS =   . Now  21 % remaining..."
[1] "Last RXN_ID =  R-MMU-6785898 : Last ARI =  0.174880529715808 : Last ECOUNT =  1 : La

[1] "Last RXN_ID =  R-MMU-2022398 : Last ARI =  0.336631806425183 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0741345520844812 : Last Uterus MISCLASS =   . Now  15.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-202721 : Last ARI =  0.499068936046279 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0539953080080619 : Last Uterus MISCLASS =   . Now  15.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-420582 : Last ARI =  0.0962284527890159 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0837933682920693 : Last Uterus MISCLASS =   . Now  15.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-2559456 : Last ARI =  0.281603672222901 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0886867215280064 : Last Uterus MISCLASS =   . Now  15.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-216723 : Last ARI =  0.321615240281456 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0801998925611349 : Last Uterus MISCLASS =   . Now  15.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-266315 : Last ARI =  0.321615240281456 : Last ECOUNT =  1 : L

[1] "Last RXN_ID =  R-MMU-111253 : Last ARI =  0.0574029245911434 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0803162228321652 : Last Uterus MISCLASS =   . Now  10.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-2213244 : Last ARI =  0.236515402539536 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0519361795176627 : Last Uterus MISCLASS =   . Now  10.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-6781899 : Last ARI =  0.174421967039958 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0996482731331574 : Last Uterus MISCLASS =   . Now  10 % remaining..."
[1] "Last RXN_ID =  R-MMU-2730863 : Last ARI =  0.0521039984737411 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0769239224913528 : Last Uterus MISCLASS =   . Now  9.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-2395849 : Last ARI =  0.178153136932765 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0961806433635673 : Last Uterus MISCLASS =   . Now  9.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1250264 : Last ARI =  0.270797856596162 : Last ECOUNT =  1 : L

[1] "Last RXN_ID =  R-MMU-444204 : Last ARI =  0.0793595360307665 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0678857759532063 : Last Uterus MISCLASS =   . Now  4.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9663511 : Last ARI =  0.289988711997475 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0712016217348055 : Last Uterus MISCLASS =   . Now  4.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-994140 : Last ARI =  0.0808616478278583 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0829570505651044 : Last Uterus MISCLASS =   . Now  4.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-6800035 : Last ARI =  0.214178123102195 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0914137041286309 : Last Uterus MISCLASS =   . Now  4.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-5661125 : Last ARI =  0.300448025651618 : Last ECOUNT =  9 : Last Lung MISCLASS =  0.0622589542547029 : Last Uterus MISCLASS =   . Now  4 % remaining..."
[1] "Last RXN_ID =  R-MMU-9822463 : Last ARI =  0.347634279116261 : Last ECOUNT =  12 : Last

In [18]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls.Rds", sep = ""))

In [19]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df.Rds", sep = ""))

                               RXN_ID     MISCLASS          ARI ECOUNT
R-MMU-450466.Brain       R-MMU-450466 0.0546182962  0.777817504     17
R-MMU-450466.Liver       R-MMU-450551 0.0435775859  0.440314259      4
R-MMU-450466.Intestine   R-MMU-450580 0.0193940592  0.440314259      4
R-MMU-450466.Eye          R-MMU-72103 0.0138511721  0.769507879     23
R-MMU-450466.Heart      R-MMU-9770119 0.0096713709  0.742627743     33
R-MMU-450466.MOE        R-MMU-9770141 0.0006756757  0.749417372     61
R-MMU-450466.Muscle     R-MMU-9770142 0.0089904493  0.755185495     67
R-MMU-450466.Lung       R-MMU-9770145 0.0124581474  0.765489633     75
R-MMU-450466.Kidney     R-MMU-9770236 0.0172344723  0.753127833     59
R-MMU-450466.Pancreas   R-MMU-6798748 0.0207498771  0.877299450     55
R-MMU-450466.Testes     R-MMU-6800434 0.0097231450  0.877974988     69
R-MMU-450466.Adipose    R-MMU-6791218 0.0214254858  0.702139054     30
R-MMU-450466.Skin       R-MMU-6791222 0.0138283345  0.813316846     36
R-MMU-

R-MMU-9770236.Testes    R-MMU-5653770 0.0048425419  0.715830948     10
R-MMU-9770236.Adipose   R-MMU-5653780 0.0193228615  0.626677052      7
R-MMU-9770236.Skin      R-MMU-5653786 0.0116974859  0.626677052      7
R-MMU-6798748.Brain     R-MMU-5653838 0.0360434130  0.700964472     11
R-MMU-6798748.Liver     R-MMU-5654985 0.0200686903  0.703393310      9
R-MMU-6798748.Intestine R-MMU-5654986 0.0048807357  0.533253210      7
R-MMU-6798748.Eye       R-MMU-5654989 0.0048807357  0.703393310      9
R-MMU-6798748.Heart     R-MMU-5655466 0.0041290716  0.450006381      6
R-MMU-6798748.MOE       R-MMU-5655481 0.0000000000  0.450006381      6
R-MMU-6798748.Muscle    R-MMU-5655835 0.0090285680  0.629030289      8
R-MMU-6798748.Lung      R-MMU-5655892 0.0055326538  0.629030289      8
R-MMU-6798748.Kidney    R-MMU-5655965 0.0131717517  0.629030289      8
R-MMU-6798748.Pancreas  R-MMU-5656105 0.0068790243  0.651454613      8
R-MMU-6798748.Testes    R-MMU-5656148 0.0055749806  0.651454613      8
R-MMU-

R-MMU-9840795.Kidney     R-MMU-917744 0.0311194466  0.810039135      7
R-MMU-9840795.Pancreas  R-MMU-9664156 0.0333021938  0.785794045      6
R-MMU-9840795.Testes    R-MMU-6785821 0.0068835484  0.427275751      3
R-MMU-9840795.Adipose   R-MMU-6786058 0.0380167831  0.472772444      3
R-MMU-9840795.Skin      R-MMU-6786070 0.0332171478  0.427275751      3
R-MMU-9840833.Brain     R-MMU-6786072 0.0934012262  0.659247907      5
R-MMU-9840833.Liver     R-MMU-6786092 0.1189917733  0.472772444      3
R-MMU-9840833.Intestine R-MMU-6786095 0.0409513270  0.659247907      5
R-MMU-9840833.Eye       R-MMU-6786096 0.0310769845  0.472772444      3
R-MMU-9840833.Heart     R-MMU-6786101 0.0242318789  0.410940303      2
R-MMU-9840833.MOE       R-MMU-6786124 0.0048423518  0.659247907      5
R-MMU-9840833.Muscle    R-MMU-9603775 0.0367172715  0.791949243      8
R-MMU-9840833.Lung      R-MMU-9603784 0.0463597633  0.791949243      8
R-MMU-9840833.Kidney    R-MMU-9603804 0.0096802370  0.772777321      7
R-MMU-

R-MMU-70420.Muscle      R-MMU-4093329 0.0408422676  0.831955242      6
R-MMU-70420.Lung        R-MMU-4093331 0.0671484810  0.831955242      6
R-MMU-70420.Kidney      R-MMU-4093332 0.0263049855  0.831955242      6
R-MMU-70420.Pancreas    R-MMU-4093336 0.0318239838  0.854366386      8
R-MMU-70420.Testes      R-MMU-4093339 0.0277370489  0.831955242      6
R-MMU-70420.Adipose     R-MMU-9696266 0.0906915633  0.867721353     23
R-MMU-70420.Skin        R-MMU-9696275 0.0359434099  0.103988657      1
R-MMU-71218.Brain       R-MMU-9014741 0.2512387301  0.269617953      1
R-MMU-71218.Liver       R-MMU-9014766 0.0699217347  0.269617953      1
R-MMU-71218.Intestine   R-MMU-9034539 0.1389739481  0.269617953      1
R-MMU-71218.Eye          R-MMU-196950 0.0608926364  0.191524733      1
R-MMU-71218.Heart       R-MMU-8952618 0.0533345894  0.831665915     31
R-MMU-71218.MOE         R-MMU-8952620 0.0325564225  0.831665915     31
R-MMU-71218.Muscle      R-MMU-8955241 0.0885352630  0.841960177     52
R-MMU-

R-MMU-1912394.Heart      R-MMU-190520 0.0435162238  0.665669275      5
R-MMU-1912394.MOE       R-MMU-2213248 0.0145325490  0.862077963     32
R-MMU-1912394.Muscle    R-MMU-2316350 0.0691658720  0.665669275      5
R-MMU-1912394.Lung      R-MMU-2467809 0.0477075039  0.841006797     64
R-MMU-1912394.Kidney    R-MMU-2467811 0.0560453133  0.841730301     63
R-MMU-1912394.Pancreas  R-MMU-2468287 0.0455861669  0.841730301     63
R-MMU-1912394.Testes    R-MMU-2484822 0.0324800444  0.841730301     63
R-MMU-1912394.Adipose    R-MMU-375302 0.0581388158  0.847923945     58
R-MMU-1912394.Skin       R-MMU-445071 0.0222050514  0.776469535     10
R-MMU-2032794.Brain     R-MMU-5610733 0.1454041478  0.537956245      3
R-MMU-2032794.Liver     R-MMU-5610767 0.1842630757  0.815080359     10
R-MMU-2032794.Intestine R-MMU-5618080 0.0950417900  0.838914714     23
R-MMU-2032794.Eye       R-MMU-5618328 0.0408048545  0.724052117      6
R-MMU-2032794.Heart     R-MMU-5618331 0.0068976605  0.730643659      6
R-MMU-

...

R-MMU-6785986.Muscle     R-MMU-975449 0.0152410399  0.082199321      1
R-MMU-6785986.Lung      R-MMU-5228521 0.0138806979  0.149885372      2
R-MMU-6785986.Kidney    R-MMU-4332356 0.0117212435  0.081650348      1
R-MMU-6785986.Pancreas  R-MMU-4411402 0.0159499861  0.081650348      1
R-MMU-6785986.Testes     R-MMU-727807 0.0083433794  0.128403305      1
R-MMU-6785986.Adipose    R-MMU-378952 0.0207404312  0.054144616      1
R-MMU-6785986.Skin       R-MMU-378956 0.0166158248  0.135836347      2
R-MMU-193068.Brain       R-MMU-378975 0.3183056593  0.649860411      3
R-MMU-193068.Liver       R-MMU-378978 0.1763702956  0.135836347      2
R-MMU-193068.Intestine   R-MMU-391377 0.1438305432  0.135836347      2
R-MMU-193068.Eye         R-MMU-391378 0.0554512617  0.072648596      2
R-MMU-193068.Heart      R-MMU-6786034 0.0532634492  0.135836347      2
R-MMU-193068.MOE          R-MMU-70342 0.0311343956  0.334272812      1
R-MMU-193068.Muscle       R-MMU-71495 0.0830468189  0.595143491      3
R-MMU-

R-MMU-114256.Heart      R-MMU-1013873 0.0311101224  0.284753967      2
R-MMU-114256.MOE         R-MMU-180047 0.0200740033  0.403872387      2
R-MMU-114256.Muscle     R-MMU-6805276 0.0650075383  0.102395611      1
R-MMU-114256.Lung       R-MMU-8868583 0.0532441100  0.102395611      1
R-MMU-114256.Kidney     R-MMU-8868589 0.0844056102  0.102395611      1
R-MMU-114256.Pancreas   R-MMU-8868677 0.0588005776  0.252102740      2
R-MMU-114256.Testes     R-MMU-9634906 0.0214387516  0.102395611      1
R-MMU-114256.Adipose    R-MMU-9634923 0.0968223909  0.102395611      1
R-MMU-114256.Skin       R-NUL-8870566 0.0332078263  0.102395611      1
R-MMU-6805507.Brain     R-MMU-1237045 0.3472684906  0.039016837      1
R-MMU-6805507.Liver     R-MMU-1237081 0.2905591079  0.039016837      1
R-MMU-6805507.Intestine  R-MMU-382560 0.1578238404  0.074891692      1
R-MMU-6805507.Eye        R-MMU-211186 0.0560446570  0.362671560      3
R-MMU-6805507.Heart      R-MMU-211190 0.0560082310  0.362671560      3
R-MMU-

R-MMU-442724.Intestine  R-MMU-8878686 0.0194226650  0.435808424      2
R-MMU-442724.Eye        R-NUL-4568946 0.0290493696  0.021104018      1
R-MMU-442724.Heart       R-MMU-111285 0.0346776844  0.104598791      1
R-MMU-442724.MOE         R-MMU-111289 0.0027542867  0.104598791      1
R-MMU-442724.Muscle       R-MMU-73815 0.0263673855  0.104598791      1
R-MMU-442724.Lung         R-MMU-73805 0.0291027775  0.250114421      1
R-MMU-442724.Kidney       R-MMU-73806 0.0415138008  0.250114421      1
R-MMU-442724.Pancreas    R-MMU-391372 0.0394401705  0.048455365      1
R-MMU-442724.Testes     R-MMU-9718360 0.0277757569  0.048455365      1
R-MMU-442724.Adipose    R-MMU-9718379 0.0498478668  0.048455365      1
R-MMU-442724.Skin       R-MMU-9718394 0.0249448350  0.048455365      1
R-MMU-9029434.Brain      R-MMU-158546 0.1309869434  0.684211709      6
R-MMU-9029434.Liver      R-MMU-174931 0.0734280847  0.706143354      9
R-MMU-9029434.Intestine R-MMU-9758661 0.0194226650  0.681217957      6
R-MMU-

R-MMU-71046.Brain       R-MMU-5218916 0.2678425347  0.227479977      2
R-MMU-71046.Liver       R-MMU-5692437 0.0871372481  0.227479977      2
R-MMU-71046.Intestine    R-MMU-110215 0.1307196903  0.057276873      1
R-MMU-71046.Eye          R-MMU-110217 0.0498960864  0.057276873      1
R-MMU-71046.Heart        R-MMU-110349 0.0484628444  0.251758178      2
R-MMU-71046.MOE         R-MMU-9674131 0.0352580901  0.201967936      2
R-MMU-71046.Muscle      R-MMU-9014627 0.0733327463  0.472372092      2
R-MMU-71046.Lung        R-MMU-1912357 0.0802203622  0.254463763      2
R-MMU-71046.Kidney      R-MMU-1980122 0.0462989904  0.254463763      2
R-MMU-71046.Pancreas    R-MMU-1980125 0.0594097135  0.254463763      2
R-MMU-71046.Testes      R-MMU-2066712 0.0373649881  0.254463763      2
R-MMU-71046.Adipose     R-MMU-9655549 0.1258239948  0.074619539      1
R-MMU-71046.Skin        R-MMU-9655598 0.0401058096  0.074619539      1
R-MMU-5688786.Brain      R-MMU-156526 0.2491742995  0.504789288      3
R-MMU-

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ]), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = GEO_model_validation_tissue_detail.vec.train
)
ggplot(pca.d) +
   geom_point(aes(x = PC1, y = PC2, colour = Section)) +
   theme_bw()
p <- plot_ly(
   x = pca.d$PC1,
   y = pca.d$PC2,
   z = pca.d$PC3,
   type = "scatter3d",
   mode = "markers",
   color = pca.d$Section,
   size = 1
)
saveWidget(p, paste0(OUT_DIR, "plotly_chart2.html"), selfcontained = TRUE)

In addition: Warning messages:
1: In RColorBrewer::brewer.pal(N, "Set2") :
  n too large, allowed maximum for palette Set2 is 8
Returning the palette you asked for with that many colors

2: In RColorBrewer::brewer.pal(N, "Set2") :
  n too large, allowed maximum for palette Set2 is 8
Returning the palette you asked for with that many colors

